In [ ]:
import requests
import time
import pandas as pd
import numpy as np
import csv
import datetime


f_name = 'live_bitcoin.csv'
keys = ["price_usd","24h_volume_usd","market_cap_usd","available_supply","total_supply","percent_change_1h","percent_change_24h","percent_change_7d"]
vals = [0]*len(keys)
df = pd.DataFrame()
count = 0
    
while True:
   
    data = requests.get("https://api.coinmarketcap.com/v1/ticker/bitcoin/").json()[0]
    df.at(count)[0] = datetime.datetime.now().strftime("%y-%m-%d-%H-%M")
    df.at(count)[1] = data['price_usd']
    df.to_csv(f_name,sep='\t')
    time.sleep(60)
#     datetime.datetime.now().strftime("%y-%m-%d-%H-%M"))
    

In [ ]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np
import datetime
from pytz import timezone
est = timezone('US/Eastern')


from twilio.rest import Client

# Your Account SID from twilio.com/console
account_sid = "AC98125a533762def33f8e326e6996b955"
# Your Auth Token from twilio.com/console
auth_token  = "2dec7e0c1517d9af09f0d4a46da6606f"

client = Client(account_sid, auth_token)



data = pd.read_csv("merged_data.csv")

datag = data[['Price','Sentiment']].groupby(data['Time']).mean()

from sklearn.preprocessing import MinMaxScaler
values = datag['Price'].values.reshape(-1,1)
sentiment = datag['Sentiment'].values.reshape(-1,1)
values = values.astype('float32')
sentiment = sentiment.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

train_size = int(len(scaled) * 0.7)
test_size = len(scaled) - train_size
train, test = scaled[0:train_size,:], scaled[train_size:len(scaled),:]
print(len(train), len(test))
split = train_size

def create_dataset(dataset, look_back, sentiment, sent=False):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        if i >= look_back:
            a = dataset[i-look_back:i+1, 0]
            a = a.tolist()
            if(sent==True):
                a.append(sentiment[i].tolist()[0])
            dataX.append(a)
            dataY.append(dataset[i + look_back, 0])
    #print(len(dataY))
    return np.array(dataX), np.array(dataY)

look_back = 2
trainX, trainY = create_dataset(train, look_back, sentiment[0:train_size],sent=True)
testX, testY = create_dataset(test, look_back, sentiment[train_size:len(scaled)], sent=True)

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

model = Sequential()
model.add(LSTM(100, input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(100))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
history = model.fit(trainX, trainY, epochs=300, batch_size=100, validation_data=(testX, testY), verbose=0, shuffle=False)

yhat = model.predict(testX)

yhat_inverse_sent = scaler.inverse_transform(yhat.reshape(-1, 1))
testY_inverse_sent = scaler.inverse_transform(testY.reshape(-1, 1))

rmse_sent = sqrt(mean_squared_error(testY_inverse_sent, yhat_inverse_sent))
print('Test RMSE: %.3f' % rmse_sent)

#import sqlite
#Enter the values for you database connection
#dsn_database = "bitcoin"         # e.g. "MySQLdbtest"
#dsn_hostname = "173.194.231.244"      # e.g.: "mydbinstance.xyz.us-east-1.rds.amazonaws.com"
#dsn_port = 3306                  # e.g. 3306 
#dsn_uid = "demo"             # e.g. "user1"
#dsn_pwd = "qwerty@123"              # e.g. "Password123"

#conn = MySQLdb.connect(host=dsn_hostname, port=dsn_port, user=dsn_uid, passwd=dsn_pwd, db=dsn_database)

#cursor=conn.cursor()


import queue 
import time

import queue
import matplotlib.pyplot as plt
true_q = queue.Queue()
pred_q = queue.Queue()
'''
fig = plt.figure()
ax = fig.add_subplot(111)
fig.show()
fig.canvas.draw()
plt.ion()
'''

def process_data(in_data):
    out_data = []
    for line in in_data:
        out_data.append(float(line.split(',')[0]))
    return np.array(out_data).reshape(-1,1)
prev = 15000
threshold = 0.05
bank = 15000
bitcoin = 0
count =0

while True:
    btc = open('live_bitcoin.csv','r')
    sent = open('live_tweet.csv','r')
    temp = pd.read_csv('predictions.csv')
    bit_data = btc.readlines()
    sent_data = sent.readlines()
    bit_data = process_data(bit_data[len(bit_data)-5:])
    print(bit_data)
    sent_data = process_data(sent_data[len(sent_data)-5:])
    print(sent_data)
    live = scaler.transform(bit_data)
    print(live)
    testX, testY = create_dataset(live, 2, sent_data, sent=True)
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    yhat = model.predict(testX)
    yhat_inverse = scaler.inverse_transform(yhat.reshape(-1, 1))
    true_q.put(bit_data[4])
    pred_q.put(yhat_inverse[0])
    val = 100*((yhat_inverse[0][0] - prev)/prev)
    print(val)
    if val > threshold:
        decision = 'Buy!!!'
        #message = client.messages.create(to="+15184234418‬", from_="+15188883052", body=decision+' - Price of Bitcoin is expected to rise.')
    elif val <-threshold:
        decision = 'Sell!!!'
        #message = client.messages.create(to="+15184234418", from_="+15188883052", body=decision+' - Price of Bitcoin is expected to drop.')
    else:
        decision = 'Hold'
    print(decision)  
    
    count = count + 1
    temp = pd.read_csv('predictions.csv','a')
    temp = pd.DataFrame(columns=["Sentiment val","time","BTC Price","Transaction type"])
    temp.loc[count] = [val,datetime.datetime.now().strftime("%y-%m-%d-%H-%M"),bit_data[4],decision]
    
    
    
    prev = yhat_inverse[0][0]
    #input_string = "INSERT INTO live_data values ({},{},{},'{}','{}');".format(yhat_inverse[0][0],bit_data[0][0],sent_data[4][0],datetime.datetime.now(tz=est).strftime('%Y-%m-%d %H:%M:%S'),decision)
    #cursor.execute(input_string)
    #conn.commit()
    time.sleep(60)
    
    






260 112
Test RMSE: 683.415
[[11215.9]
 [11215.9]
 [11215.9]
 [11222.1]
 [11222.1]]
[[0.06213259]
 [0.06213259]
 [0.06213259]
 [0.06300769]
 [0.06300769]]
[[1.57372339]
 [1.57372339]
 [1.57372339]
 [1.57465157]
 [1.57465157]]
-51.234882812500004
Sell!!!


In [ ]:
trace1 = go.Scatter(
                    x=temp['time'], y=temp[''], # Data
                    mode='lines', name='logx' # Additional options
                   )
trace2 = go.Scatter(x=df['x'], y=df['sinx'], mode='lines', name='sinx' )
trace3 = go.Scatter(x=df['x'], y=df['cosx'], mode='lines', name='cosx')

layout = go.Layout(title='Simple Plot from csv data',
                   plot_bgcolor='rgb(230, 230,230)')

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)

# Plot data in the notebook
py.iplot(fig, filename='simple-plot-from-csv')